In [36]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_distances



In [37]:
#!{sys.executable} -m pip install pandas numpy scikit-learn

In [38]:
def random_data_gen(num_rows = 1000, n_layer_1 = 3, n_layer_2 = 20, start_dt = datetime(2020, 1, 1), end_dt = datetime(2025, 1, 1) ):
    id1 = ["L1_"+str(x) for x in np.random.randint(n_layer_1, size=num_rows)]
    id2 = ["L2_"+str(x) for x in np.random.randint(n_layer_2, size=num_rows)]
    start_u = start_dt.timestamp()
    end_u = end_dt.timestamp()
    random_ts = np.random.uniform(start_u, end_u, num_rows)
    dt = sorted([datetime.fromtimestamp(ts) for ts in random_ts])

    df = pd.DataFrame({
        'L1_id': id1,
        'L2_id': id2,
        'timestamp': dt
    })
    return df   

In [39]:
df = random_data_gen()
df


,L1_id,L2_id,timestamp
0,L1_1,L2_19,2020-01-01 07:24:37.958029
1,L1_1,L2_19,2020-01-01 09:04:05.790815
2,L1_2,L2_14,2020-01-01 14:34:08.441235
3,L1_1,L2_6,2020-01-07 09:05:17.870064
4,L1_1,L2_6,2020-01-08 02:20:18.024705
...,...,...,...
995,L1_1,L2_7,2024-12-26 05:58:13.137806
996,L1_0,L2_8,2024-12-26 09:50:40.447717
997,L1_1,L2_0,2024-12-29 00:54:53.917082
998,L1_0,L2_17,2024-12-30 21:25:14.212302


In [40]:
df['timeframe'] = df['timestamp'].dt.to_period('M').apply(lambda r: str(r.start_time)+" to " + str(r.end_time))
df

,L1_id,L2_id,timestamp,timeframe
0,L1_1,L2_19,2020-01-01 07:24:37.958029,2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...
1,L1_1,L2_19,2020-01-01 09:04:05.790815,2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...
2,L1_2,L2_14,2020-01-01 14:34:08.441235,2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...
3,L1_1,L2_6,2020-01-07 09:05:17.870064,2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...
4,L1_1,L2_6,2020-01-08 02:20:18.024705,2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...
...,...,...,...,...
995,L1_1,L2_7,2024-12-26 05:58:13.137806,2024-12-01 00:00:00 to 2024-12-31 23:59:59.999...
996,L1_0,L2_8,2024-12-26 09:50:40.447717,2024-12-01 00:00:00 to 2024-12-31 23:59:59.999...
997,L1_1,L2_0,2024-12-29 00:54:53.917082,2024-12-01 00:00:00 to 2024-12-31 23:59:59.999...
998,L1_0,L2_17,2024-12-30 21:25:14.212302,2024-12-01 00:00:00 to 2024-12-31 23:59:59.999...


In [41]:
grouped_df = df.groupby(['L1_id','L2_id','timeframe']).size().reset_index(name='counts')
grouped_df

,L1_id,L2_id,timeframe,counts
0,L1_0,L2_0,2020-05-01 00:00:00 to 2020-05-31 23:59:59.999...,1
1,L1_0,L2_0,2020-08-01 00:00:00 to 2020-08-31 23:59:59.999...,1
2,L1_0,L2_0,2021-04-01 00:00:00 to 2021-04-30 23:59:59.999...,1
3,L1_0,L2_0,2021-12-01 00:00:00 to 2021-12-31 23:59:59.999...,1
4,L1_0,L2_0,2022-03-01 00:00:00 to 2022-03-31 23:59:59.999...,1
...,...,...,...,...
861,L1_2,L2_9,2023-12-01 00:00:00 to 2023-12-31 23:59:59.999...,1
862,L1_2,L2_9,2024-01-01 00:00:00 to 2024-01-31 23:59:59.999...,1
863,L1_2,L2_9,2024-03-01 00:00:00 to 2024-03-31 23:59:59.999...,1
864,L1_2,L2_9,2024-04-01 00:00:00 to 2024-04-30 23:59:59.999...,1


In [42]:
# Pivot grouped_df so that L2_id becomes columns
pivot_df = grouped_df.pivot(index=['L1_id', 'timeframe'], columns='L2_id', values='counts').fillna(0)
pivot_df

L2_id                                                     L2_0  L2_1  L2_10  \
L1_id timeframe                                                               
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...   0.0   0.0    0.0   
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...   0.0   1.0    0.0   
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...   0.0   0.0    0.0   
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...   0.0   0.0    0.0   
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...   1.0   1.0    0.0   
...                                                        ...   ...    ...   
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...   1.0   0.0    1.0   
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...   1.0   0.0    0.0   
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999...   3.0   0.0    3.0   
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...   1.0   0.0    0.0   
      2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...   0.0   0.0    1.0   

L2_id                                                     L2_11  L2_12  L2_13  \
L1_id timeframe                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...    0.0    0.0    0.0   
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...    0.0    0.0    0.0   
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...    0.0    0.0    0.0   
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...    0.0    0.0    1.0   
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...    0.0    0.0    1.0   
...                                                         ...    ...    ...   
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...    0.0    0.0    0.0   
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...    0.0    0.0    0.0   
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999...    0.0    0.0    0.0   
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...    0.0    0.0    0.0   
      2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...    1.0    1.0    0.0   

L2_id                                                     L2_14  L2_15  L2_16  \
L1_id timeframe                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...    1.0    0.0    0.0   
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...    1.0    0.0    0.0   
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...    0.0    0.0    0.0   
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...    0.0    0.0    0.0   
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...    0.0    0.0    0.0   
...                                                         ...    ...    ...   
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...    0.0    0.0    1.0   
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...    0.0    2.0    1.0   
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999...    0.0    0.0    2.0   
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...    0.0    1.0    0.0   
      2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...    0.0    0.0    0.0   

L2_id                                                     L2_17  L2_18  L2_19  \
L1_id timeframe                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...    0.0    1.0    0.0   
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...    0.0    1.0    2.0   
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...    1.0    0.0    1.0   
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...    0.0    0.0    0.0   
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...    0.0    1.0    0.0   
...                                                         ...    ...    ...   
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...    0.0    1.0    0.0   
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...    0.0    0.0    0.0   
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999.

In [ ]:
tmp_index_tf = []

relative_cosine_dist_matrices = pd.DataFrame()

for tf in pivot_df.index.levels[1].unique():
    tmp_df = pivot_df.xs(tf, level='timeframe')
    tmp_cosine_dist_matrix = pd.DataFrame(cosine_distances(tmp_df), index=tmp_df.index, columns=tmp_df.index)
    relative_cosine_dist_matrices = pd.concat([relative_cosine_dist_matrices, tmp_cosine_dist_matrix], axis=0)
    tmp_index_tf.extend([tf] * len(tmp_cosine_dist_matrix))

relative_cosine_dist_matrices['timeframe'] = tmp_index_tf
relative_cosine_dist_matrices.set_index(['timeframe'], append=True, inplace=True)
relative_cosine_dist_matrices

L1_id                                                         L1_0      L1_1  \
L1_id timeframe                                                                
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.000000  0.773545   
L1_1  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.773545  0.000000   
L1_2  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.634852  0.875965   
L1_0  2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...  0.000000  0.803884   
L1_1  2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...  0.803884  0.000000   
...                                                            ...       ...   
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...  1.000000  0.000000   
L1_2  2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...  1.000000  1.000000   
L1_0  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  0.000000  1.000000   
L1_1  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  1.000000  0.000000   
L1_2  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  0.776393  1.000000   

L1_id                                                         L1_2  
L1_id timeframe                                                     
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.634852  
L1_1  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.875965  
L1_2  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...  0.000000  
L1_0  2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...  1.000000  
L1_1  2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...  0.683772  
...                                                            ...  
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...  1.000000  
L1_2  2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...  0.000000  
L1_0  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  0.776393  
L1_1  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  1.000000  
L1_2  2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...  0.000000  

[180 rows x 3 columns]

In [ ]:
#df.set_index(pd.Index(['2023-01-01'] * len(df), name='date'), append=True)

L1_id,L1_0,L1_1,L1_2
L1_id,,,
L1_0,0.000000,0.773545,0.634852
L1_1,0.773545,0.000000,0.875965
L1_2,0.634852,0.875965,0.000000


In [43]:
cosine_dist_matrix = pd.DataFrame(cosine_distances(pivot_df), index=pivot_df.index, columns=pivot_df.index)
cosine_dist_matrix

L1_id                                                                                                    L1_0  \
timeframe                                                2020-01-01 00:00:00 to 2020-01-31 23:59:59.999999999   
L1_id timeframe                                                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...                                           0.000000     
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...                                           0.773545     
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...                                           1.000000     
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...                                           1.000000     
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...                                           0.817426     
...                                                                                                     ...     
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...                                           0.817426     
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...                                           1.000000     
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999...                                           0.833333     
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...                                           0.591752     
      2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           1.000000     

L1_id                                                                                                          \
timeframe                                                2020-02-01 00:00:00 to 2020-02-29 23:59:59.999999999   
L1_id timeframe                                                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...                                           0.773545     
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...                                           0.000000     
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...                                           0.306625     
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...                                           0.679744     
      2020-05-01 00:00:00 to 2020-05-31 23:59:59.9999...                                           0.503861     
...                                                                                                     ...     
L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.9999...                                           0.751931     
      2024-09-01 00:00:00 to 2024-09-30 23:59:59.9999...                                           1.000000     
      2024-10-01 00:00:00 to 2024-10-31 23:59:59.9999...                                           0.886772     
      2024-11-01 00:00:00 to 2024-11-30 23:59:59.9999...                                           1.000000     
      2024-12-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.861325     

L1_id                                                                                                          \
timeframe                                                2020-03-01 00:00:00 to 2020-03-31 23:59:59.999999999   
L1_id timeframe                                                                                                 
L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.9999...                                           1.000000     
      2020-02-01 00:00:00 to 2020-02-29 23:59:59.9999...                                           0.306625     
      2020-03-01 00:00:00 to 2020-03-31 23:59:59.9999...                                           0.000000     
      2020-04-01 00:00:00 to 2020-04-30 23:59:59.9999...                                           0.711325     
      2020-05-01 00:00:00 to 2020-05-31 23:

L1_id     L1_id                                          timeframe  \
timeframe                                                            
0          L1_0  2020-01-01 00:00:00 to 2020-01-31 23:59:59.999...   
1          L1_0  2020-02-01 00:00:00 to 2020-02-29 23:59:59.999...   
2          L1_0  2020-03-01 00:00:00 to 2020-03-31 23:59:59.999...   
3          L1_0  2020-04-01 00:00:00 to 2020-04-30 23:59:59.999...   
4          L1_0  2020-05-01 00:00:00 to 2020-05-31 23:59:59.999...   
..          ...                                                ...   
175        L1_2  2024-08-01 00:00:00 to 2024-08-31 23:59:59.999...   
176        L1_2  2024-09-01 00:00:00 to 2024-09-30 23:59:59.999...   
177        L1_2  2024-10-01 00:00:00 to 2024-10-31 23:59:59.999...   
178        L1_2  2024-11-01 00:00:00 to 2024-11-30 23:59:59.999...   
179        L1_2  2024-12-01 00:00:00 to 2024-12-31 23:59:59.999...   

L1_id                                                     L1_0  \
timeframe 2020-01-01 00:00:00 to 2020-01-31 23:59:59.999999999   
0                                                   0.000000     
1                                                   0.773545     
2                                                   1.000000     
3                                                   1.000000     
4                                                   0.817426     
..                                                       ...     
175                                                 0.817426     
176                                                 1.000000     
177                                                 0.833333     
178                                                 0.591752     
179                                                 1.000000     

L1_id                                                           \
timeframe 2020-02-01 00:00:00 to 2020-02-29 23:59:59.999999999   
0                                                   0.773545     
1                                                   0.000000     
2                                                   0.306625     
3                                                   0.679744     
4                                                   0.503861     
..                                                       ...     
175                                                 0.751931     
176                                                 1.000000     
177                                                 0.886772     
178                                                 1.000000     
179                                                 0.861325     

L1_id                                                           \
timeframe 2020-03-01 00:00:00 to 2020-03-31 23:59:59.999999999   
0                                                   1.000000     
1                                                   0.306625     
2                                                   0.000000     
3                                                   0.711325     
4                                                   0.776393     
..                                                       ...     
175                                                 0.776393     
176                                                 1.000000     
177                                                 0.897938     
178                                                 1.000000     
179                                                 1.000000     

L1_id                                                           \
timeframe 2020-04-01 00:00:00 to 2020-04-30 23:59:59.999999999   
0                                                   1.000000     
1                                                   0.679744     
2                                                   0.711325     
3                                                   0.000000     
4                                                   0.483602     
..                                                